In [29]:
import sympy as sp
import numpy as np

# Task 1

Derive dynamic model for your robot model using the Euler-Lagrange approach

In [30]:
from collections import namedtuple

Angles = namedtuple('Angles', ['theta_1', 'theta_2', 'theta_3', 'theta_4', 'theta_5', 'theta_6'])

Configuration = namedtuple('Configuration', ['d_1', 'd_2', 'd_3', 'd_4', 'd_5'])
general_configuration = Configuration(*[sp.Symbol(f'd_{i}') for i in range(1, 6)])

t = sp.Symbol('t')
general_angles = Angles(*[sp.Function(f'q_{i}')(t) for i in range(1, 7)])


def get_dh_parameters(angles: Angles):
    dh_parameters = [
        # a                         d                                                       alpha       theta
        (0,                         general_configuration.d_1,                              sp.pi / 2,  angles.theta_1),
        (general_configuration.d_2, 0,                                                      0,          angles.theta_2),
        (0,                         0,                                                      -sp.pi / 2, angles.theta_3 - sp.pi / 2),
        (0,                         general_configuration.d_3 + general_configuration.d_4,  sp.pi / 2,  angles.theta_4 - sp.pi / 2),
        (0,                         0,                                                      -sp.pi / 2, angles.theta_5),
        (0,                         general_configuration.d_5,                              0,          angles.theta_6)
    ]
    return dh_parameters


In [31]:
class Rotation:
    @staticmethod
    def around_z(_theta):
        return sp.Matrix([[sp.cos(_theta),  -sp.sin(_theta),    0,  0],
                          [sp.sin(_theta),  sp.cos(_theta),     0,  0],
                          [0,                0,                 1,  0],
                          [0,               0,                  0,  1]])
    
    @staticmethod
    def around_x(_theta):
        return sp.Matrix([[1,   0,              0,                  0],
                          [0,   sp.cos(_theta), -sp.sin(_theta),    0],
                          [0,   sp.sin(_theta), sp.cos(_theta),     0],
                          [0,   0,              0,                  1]])


class Translation:
    @staticmethod
    def on_z(distance):
        return sp.Matrix([[1, 0, 0, 0],
                          [0, 1, 0, 0],
                          [0, 0, 1, distance],
                          [0, 0, 0, 1]])
    
    @staticmethod
    def on_x(distance):
        return sp.Matrix([[1, 0, 0, distance],
                          [0, 1, 0, 0],
                          [0, 0, 1, 0],
                          [0, 0, 0, 1]])

In [32]:
from operator import mul
from functools import reduce


def dh_transform(a, d, alpha, theta):
    rotation_theta = Rotation.around_z(theta)
    translation_d = Translation.on_z(d)
    translation_a = Translation.on_x(a)
    rotation_alpha = Rotation.around_x(alpha)

    return reduce(mul, [rotation_theta, translation_d, translation_a, rotation_alpha])

In [33]:
from itertools import starmap, accumulate


def forward_kinematics(angles: Angles):
    transforms = starmap(dh_transform, get_dh_parameters(angles))

    base_frame = sp.eye(4)
    frames_history = list(accumulate(transforms, func=mul, initial=base_frame))

    return frames_history

In [34]:
frames = forward_kinematics(general_angles)

In [35]:
com_dh = [
    [0, general_configuration.d_1 / 3, 0, general_angles.theta_1],
    [general_configuration.d_2 / 3, 0, 0, general_angles.theta_2],
    [general_configuration.d_3 / 3, 0, 0, general_angles.theta_3],
    [0, general_configuration.d_3 + general_configuration.d_4 / 3, 0, general_angles.theta_4],
    [0, general_configuration.d_5 / 3, 0, general_angles.theta_6]
]

com_frames = [frame * dh_transform(*dh) for frame, dh in zip(frames, com_dh)]

In [36]:
inertia_tensors = [sp.Matrix([[sp.Symbol(f'I_{{{first}{second}_{i}}}') for second in ('x', 'y', 'z')] for first in ('x', 'y', 'z')]) for i in range(1, len(com_frames))]
inertia_tensors = [tensor.replace(tensor[1, 0], tensor[0, 1]).replace(tensor[2, 0], tensor[0, 2]).replace(tensor[2, 1], tensor[1, 2]) for tensor in inertia_tensors]

In [37]:
link_masses = [sp.Symbol(f'm_{i}') for i in range(1, len(com_frames) + 1)]

In [38]:
linear_jacobians = []
for frame in com_frames:
    jacobian = sp.zeros(3, len(com_frames))

    for i, q in enumerate(general_angles[:len(com_frames)]):
        jacobian[:, i] = frame[:3, 3].diff(q)

    jacobian.simplify()

    linear_jacobians.append(jacobian)


In [39]:
angular_jacobians = []
for frame in com_frames:
    jacobian = sp.zeros(3, len(com_frames))

    for i, q in enumerate(general_angles[:len(com_frames)]):
        jacobian[:, i] = frame[:3, 2].diff(q)
    
    jacobian.simplify()

    angular_jacobians.append(jacobian)

In [40]:
inertial_matrix = sp.zeros(len(com_frames))

for i, (mass, inertia, linear_jacobian, angular_jacobian, frame) in enumerate(zip(link_masses, inertia_tensors, linear_jacobians, angular_jacobians, com_frames)):
    inertial_matrix += mass * linear_jacobian.T * linear_jacobian
    inertial_matrix += angular_jacobian.T * frame[:3, :3] * inertia * frame[:3, :3].T * angular_jacobian

inertial_matrix.simplify()

In [41]:
inertial_matrix.is_symmetric()

True

In [42]:
from itertools import product

coriolis = sp.zeros(len(com_frames))

for i, j in product(range(len(com_frames)), range(len(com_frames))):
    for k in range(len(com_frames)):
        q_dot = general_angles[k].diff(t)
        c_ijk = 0.5 * (inertial_matrix[i, j].diff(general_angles[k]) + inertial_matrix[i, k].diff(general_angles[j]) - inertial_matrix[j, k].diff(general_angles[i]))
        coriolis[i, j] += c_ijk * q_dot

In [44]:
from functools import reduce
from operator import add

acceleration = sp.Matrix([0, 0, -9.80665])

gravity_vector = reduce(add, [mass * jacobian.T * acceleration for mass, jacobian in zip(link_masses, linear_jacobians)])

In [45]:
# given in grams * square millimeters
inertia_tensors_values = np.array([[[127315.49, 0.0, 0.0],
                                    [0.0, 17300.01, 0.0], 
                                    [0.0, 0.0, 127315.49]]] * len(com_frames))
inertia_tensors_values *= 1e-9 

# given in grams
link_masses_values = np.array([111.83] * len(com_frames))
link_masses_values *= 1e-3

In [46]:
configuration = Configuration(1., 3., 2., 1., 1.)

In [47]:
# substituting values for inertia tensors
for general_inertia_tensor, inertia_tensor_values in zip(inertia_tensors, inertia_tensors_values):
    for i, j in product(range(general_inertia_tensor.shape[0]), range(general_inertia_tensor.shape[1])):
        inertial_matrix = inertial_matrix.subs(general_inertia_tensor[i, j], inertia_tensor_values[i, j])
        coriolis = coriolis.subs(general_inertia_tensor[i, j], inertia_tensor_values[i, j])

In [48]:
# substituting values for masses
for general_mass, mass_value in zip(link_masses, link_masses_values):
    inertial_matrix = inertial_matrix.subs(general_mass, mass_value)
    coriolis = coriolis.subs(general_mass, mass_value)
    gravity_vector = gravity_vector.subs(general_mass, mass_value)

In [49]:
# substituting values for links lengths
for general_length, length_value in zip(general_configuration, configuration):
    inertial_matrix = inertial_matrix.subs(general_length, length_value)
    coriolis = coriolis.subs(general_length, length_value)
    gravity_vector = gravity_vector.subs(general_length, length_value)

In [55]:
angular_velocities = [q.diff(t) for q in general_angles]

In [56]:
get_inertial_matrix = sp.lambdify(general_angles, inertial_matrix, 'numpy')
get_coriolis_matrix = sp.lambdify([*general_angles, *angular_velocities], coriolis, 'numpy')
get_gravity_vector = sp.lambdify(general_angles, gravity_vector, 'numpy')

# Task 2
Drive the robot joints between $[0,\pi]$

With zero velocity and acceleration (gravity torques)

In [ ]:
points = [np.linspace(0, np.pi) for _ in range(len(general_angles))]